In [85]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [106]:
# parameters
p = 4 # nr of random variables
n = 1000 # nr of realizations
mean = 0
sigma = 1.1

Covariance matrix is given by $ \textbf{C} = \hat{\lambda}^* \textbf{T} + (1-\hat{\lambda}^*)\textbf{S} $, where $\hat{\lambda}^*$ is the estimated optimal shrinkage intensity, $\textbf{T}$ is the target matrix which we take to be the identity, and $\textbf{S}$ is the empirically estimated covariance matrix.

We run a 100 simulations to find the shrinkage intensity.

In [107]:
T = np.identity(p) # Target matrix
C_true = sigma**2 * np.identity(p) # True covariance

In [108]:
lmbdas_est = []
S_sum = np.zeros((p, p))
m = 100 # nr of realizations
for _ in range(m):
    X = np.random.normal(mean, sigma, size=(p, n))
    
    x_mean = np.sum(X, axis=1)/n # Find mean of each row
    x_mean_M = np.tile(x_mean, (n, 1)).T # Repeat mean values as columns in a p x n matrix
    Y = X - x_mean_M
    
    W = []
    # Generate W array (which is 3D) of size (n, p, p), order of indices (k, i, j)
    for k in range(n):
        w = Y[:,k]
        W.append(np.outer(w, w))
    W_mean = np.sum(W, axis=0)/n
    
    # Emperically estimated covariance matrix
    S = n / (n-1) * W_mean
    S_sum += S
    
    W_mean_rep = np.tile(W_mean, (n, 1, 1))
    V = W - W_mean_rep
    Var = n / (n-1)**3 * np.sum(V**2, axis=0)
    
    lmbda_est = np.sum(Var) / np.sum((T-S)**2)
    lmbdas_est.append(lmbda_est)


S_avg = S_sum / m

lmbda_est = np.average(lmbdas_est)
print(f"Optimal value for shrinkage intensity: {lmbda_est}")
lmbda_est_se = np.std(lmbdas_est) #/ np.sqrt(len(lmbdas_est))
print(f"Standard deviation: {lmbda_est_se}")

print(S_avg)

Optimal value for shrinkage intensity: 0.1502633923559731
Standard deviation: 0.03102165296773172
[[ 1.20784247e+00 -3.96797596e-04  1.01993630e-03 -2.63181883e-03]
 [-3.96797596e-04  1.21292154e+00 -3.62352144e-03  1.98208851e-03]
 [ 1.01993630e-03 -3.62352144e-03  1.19726399e+00 -2.45818877e-03]
 [-2.63181883e-03  1.98208851e-03 -2.45818877e-03  1.21335746e+00]]


In [109]:
C_shrinkage = lmbda_est*T + (1-lmbda_est)*S_avg

print(C_shrinkage)

MSE_MC = np.trace((S_avg - C_true)@(S_avg - C_true).T)
print(MSE_MC)

[[ 1.17661136e+00 -3.37173443e-04  8.66677212e-04 -2.23635281e-03]
 [-3.37173443e-04  1.18092722e+00 -3.07903881e-03  1.68425317e-03]
 [ 8.66677212e-04 -3.07903881e-03  1.16762243e+00 -2.08881299e-03]
 [-2.23635281e-03  1.68425317e-03 -2.08881299e-03  1.18129764e+00]]
0.00024911972083760544
